# Niche reconstruction and spatial domain detection

In this notebook we will cover:

1. Graph construction and analysis of spatial transcriptomics data using Squidpy
2. CellCharter

In [ ]:
# Data analysis and ML imports
import pandas as pd
import matplotlib.pyplot as plt

# single-cell imports
import squidpy as sq
import scanpy as sc

from pathlib import Path
import os

import warnings
warnings.filterwarnings("ignore")

## Dataset

We will use the Xenium AD dataset from the previous notebooks here.

As a reminder the dataset consists of 6 coronal mouse brain slices from 2 different conditions (wildtype - ctrl vs TgCRND8 - AD) across 3 timepoints. We have annotated cell types that are available in  `adata.obs['cell_types']`. Please note that these annotation are not perfect. For example, there are quite some cells that could not be assigned to a cell type (NaN or "unkown"). We used leiden clustering and marker genes reported in [this](https://pages.10xgenomics.com/rs/446-PBO-704/images/10x_LIT000210_App-Note_Xenium-In-Situ_Letter_Digital.pdf) document for the annotation (check corresponding jupyter notebook). 

In this practical we aim to understand the differences of the mouse brain between the two conditions and across the timepoints using niches and spatial domains.

In [ ]:
PATH = "/Users/francesca.drummer/Documents/1_Projects/GSCN workshop/"

In [ ]:
# load adata
adata = sc.read_h5ad(Path(PATH, 'xenium_mouse_ad_annotated_rotated_domain.h5ad'))
adata

In [ ]:
# Creating a DataFrame from 'split', 'fov', and 'condition'
df = adata.obs[['condition', 'time', 'batch_key']]
value_counts = pd.DataFrame(df.values, columns=df.columns).value_counts()
print(value_counts)

In [ ]:
adata.obs['cell_types']

## 1. Cell neighborhood detection via graph construction

Spatial transcriptomics data can be represented as graphs with cells as nodes and edges as relations. Depending on the technology (imaging-based or sequencing-based) different assumptions can be made for the graph structure. 
Here we will explore graph construction and cell neighborhood analysis using the squidpy `sq.gr.spatial_neighbors` module on imaging-based data. 

*Information for graph reconstruciton for sequencing-based data can be found [here](https://squidpy.readthedocs.io/en/stable/notebooks/examples/graph/compute_spatial_neighbors.html).*

For image-based technologies we construct a graph with `coord_type=generic`, meaning that the nodes / cells will preserve their spatial location and will not be re-arranged in a spatial grid. For generic graph approaches we can choose to set a fixed radius `radius` or number of neighbors to connect to (`n_neighs`).

Below we try both graph construction methods and give them a different `key` to add to the `adata` object.

In [ ]:
if 'cell_types_colors' in adata.uns:
    del adata.uns['cell_types_colors']
    
sq.gr.spatial_neighbors(adata, n_neighs=5, coord_type="generic", key_added = 'neighs_based_spatial')
sq.pl.spatial_scatter(
    adata,
    shape=None,
    library_key = 'sample',
    color=["cell_types"],
    connectivity_key="neighs_based_spatial_connectivities",
    title=adata.obs['sample'].cat.categories,
    ncols=3,
    size = 10
)

In [ ]:
sq.gr.spatial_neighbors(adata, radius=0.2, coord_type="generic", key_added = 'radius_based_spatial')
sq.pl.spatial_scatter(
    adata,
    shape=None,
    library_key = 'sample', 
    color="cell_types",
    connectivity_key="radius_based_spatial_connectivities",
    title=adata.obs['sample'].cat.categories,
    ncols=3,
    size=10,
)

First differences across the graphs can be observed by visual inspection, e.i. the cells that are further away from the clear brain structures are connected in the clostest neighbor approach but unconnected in the radius-based approach. Some other differences are harder to observe like dependencies in the more dense connected regions. For this Squidpy provides a number of statistics to better the differences between the connectivities of cells.

Using the cell type information we can formulate some hypothesis from the data.

To perform downstream analysis you require a graph structure or radius that connects the cells of interest. For example, if you are interested in the communication between Microglia and Oligodendrocytes you need to set a radius such that the cell typer connections exists.

<span style="color: red;">**Task 1:** Compute the [interaction matrix](https://squidpy.readthedocs.io/en/stable/notebooks/examples/graph/compute_interaction_matrix.html) to understand differences between neighborhood and radius based graphs. Find an appropriate number of neighbors or radius that include sufficient connections between your cell types of interest.</span>

Example questions to answer could be: What are the average number of connections per cell type? Which cell types tend to cluster together? 

In [ ]:
sq.gr.interaction_matrix(adata, cluster_key="cell_types", connectivity_key="neighs_based_spatial")
sq.pl.interaction_matrix(adata, cluster_key="cell_types", connectivity_key="neighs_based_spatial")

In [ ]:
#TODO: Interaction matrix for radius based graph

<span style="color: red;">**Task 2:** Try out some analysis from Squidpy to identify distinctions between the graphs e.i. using the [centrality score](https://squidpy.readthedocs.io/en/stable/notebooks/examples/graph/compute_centrality_scores.html) and [neighborhood enrichment](https://squidpy.readthedocs.io/en/stable/notebooks/examples/graph/compute_nhood_enrichment.html). </span>

In [ ]:
sq.gr.centrality_scores(adata, cluster_key = "cell_types", connectivity_key = "neighs_based_spatial")
sq.pl.centrality_scores(adata, cluster_key = "cell_types")

In [ ]:
sq.gr.nhood_enrichment(adata, cluster_key="cell_types", library_key = 'condition', connectivity_key = "radius_based_spatial")
sq.pl.nhood_enrichment(
    adata, cluster_key="cell_types", method="average", figsize=(5, 5)
) 

## 2. Spatial Domain detection with CellCharter

In [ ]:
import scvi
import scanpy as sc
from pathlib import Path
import matplotlib.pyplot as plt
import squidpy as sq
import numpy as np
import cellcharter as cc
import os
import logging
logger = logging.getLogger('pytorch_lightning.utilities.rank_zero')
logger.setLevel(logging.ERROR)

In [ ]:
scvi.settings.seed = 12345
scvi.settings.num_threads = 2

There may be cells with very low counts. We will filter them out.

In [ ]:
sc.pp.filter_cells(adata, min_counts=3)

### 2.1 Dimensionality reduction

First, we need to run the dimensionality reduction. We will use [scVI](https://docs.scvi-tools.org/en/latest/api/reference/scvi.model.SCVI.html) for this.

Make sure that `adata.X` contains count data, as it is required by scVI. <br>
In some tutorials you will see that the count data is stored in an AnnData layer called `counts`. Here we will use the `X` layer.

In [ ]:
scvi.model.SCVI.setup_anndata(adata)

In [ ]:
LOAD_MODEL = False

We set the parameters of the neural network. Here we use 1 layer and an embedding size of 10.<br>
These are very common parameters, but you can play with them to see how they affect the results.

In [ ]:
if LOAD_MODEL:
    model = scvi.model.SCVI.load(os.path.join(PATH, 'scvi_model'), adata=adata)
else:
    model = scvi.model.SCVI(
        adata,
        n_layers=1,
        n_latent=10,
        use_layer_norm="both",
        use_batch_norm="none",
    )
    model.train(early_stopping=True, enable_progress_bar=True, max_epochs=10)

To make sure that the training has converged, we can plot the training history.<br>
Here we is important to focus on the (validation) reconstruction loss. This shows how well the model is able to reconstruct the original data.

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(
    model.history[f"reconstruction_loss_train"],
    label="train",
    color="darkgreen",
    linewidth=1.25
)
plt.plot(
    model.history[f"reconstruction_loss_validation"],
    label="validation",
    color="firebrick",
    linewidth=1.25
    )
plt.legend()
plt.title("reconstruction_loss")  
plt.tight_layout()

We extract the latent representation of the data.

In [ ]:
adata.obsm['X_scVI'] = model.get_latent_representation(adata).astype(np.float32)

### 2.2 Neighborhood aggregation

Until now, all the analyses have been done ignoring the spatial information. <br>
Now we will use the spatial information to perform the clustering.

First, we need to create a network where cells are connected if they are close to each other using the Delaunay triangulation.

In [ ]:
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='generic')

In [ ]:
sq.pl.spatial_scatter(
    adata, 
    shape=None, 
    library_key='sample',
    library_id=adata.obs['sample'].cat.categories[0],
    color="sample", 
    size=1, 
    figsize=(10,10),
    connectivity_key="spatial_connectivities",
    ncols=1
)

As you can see, the Delaunay triangulation generates very long edges for a few cells. <br>
The most appropriate approach would be to estimate the most biologically relevant distance between cells and use it to remove the edges longer than that distance.

A quick alternative solution is that, since those long edges are sort of outlines, the measure the 99th percentile of the edge lengths and remove the edges longer than that distance. <br>
This process will lead to some isolated cells, but that's not a problem.


In [ ]:
sq.gr.spatial_neighbors(adata, library_key='sample', coord_type='generic', percentile=99)

In [ ]:
sq.pl.spatial_scatter(
    adata, 
    shape=None, 
    library_key='sample',
    library_id=adata.obs['sample'].cat.categories[0],
    color="sample", 
    size=1, 
    figsize=(10,10),
    connectivity_key="spatial_connectivities",
    ncols=1
)

We construct the neighborhood aggregated representation by combining every cell's features with the ones of first 3 layers of neighbors.<br>
We will use the scVI latent representation as features and the new features will be stored in `adata.obsm['X_cellcharter']`.

In [ ]:
cc.gr.aggregate_neighbors(adata, n_layers=3, use_rep='X_scVI', out_key='X_cellcharter_temp', sample_key='sample')

<span style="color: red;">**Task 3:** Given that `X_scVI` contains 10 features, how many features does `X_cellcharter` contain?<br>
Guess the answer and then write the code to check it.
</span>

### 2.3 Clustering

Finally, let's cluster the cells and find their spatial domains.<br>
We will use 18 clusters, we will see later why this is a good choice.

In [ ]:
gmm = cc.tl.Cluster(n_clusters=18, random_state=12345)
gmm.fit(adata, use_rep='X_cellcharter_temp')
adata.obs['spatial_domain_temp'] = gmm.predict(adata, use_rep='X_cellcharter_temp')

We can now plot domains and cell types back to back

In [ ]:
if 'spatial_domain_temp_colors' in adata.uns:
    del adata.uns['spatial_domain_temp_colors']

sq.pl.spatial_scatter(
    adata, 
    shape=None, 
    library_key='sample', 
    color=["spatial_domain_temp", "cell_types"], 
    size=1,
    figsize=(10,10),
    title=np.repeat(adata.obs['sample'].cat.categories, 2),
    ncols=2
)

### 2.4 Downstream analyses

The last section was to show how CellCharter can be used to obtain spatial domains.

However, to have an easier discussion and interpretation of the results, we want everyone to have the same results.

Therefore, we will use the spatial domains computed in advance using the `ClusterAutoK` model that also runs the stability analysis to find the optimal number of clusters.
The model has been computed using 3 layers of neighbors (it took 25 minutes on a GPU).

We have to:
1. load the features from the pretrained scvi model to generate the same features used by fitted `ClusterAutoK`
2. aggregate the neighborhood features using the same number of layers as used by `ClusterAutoK` (3 layers of neighbors)
3. load the `ClusterAutoK` model
4. plot the stability curve
5. look for the peak(s) to identify the optimal number of clusters.

In [ ]:
model = scvi.model.SCVI.load(os.path.join(PATH, 'scvi_model'), adata=adata)
adata.obsm['X_scVI'] = model.get_latent_representation(adata).astype(np.float32)
cc.gr.aggregate_neighbors(adata, n_layers=3, use_rep='X_scVI', out_key='X_cellcharter', sample_key='sample')

autok = cc.tl.ClusterAutoK.load(Path(PATH, 'autok_l3'))
cc.pl.autok_stability(autok)

# If it takes too long also in this case, you can load the domain labels from the data folder
# adata.obs['spatial_domain_18'] = pd.read_csv(Path(PATH, 'spatial_domains_cellcharter.csv'), index_col=0)['spatial_domain_18']
# adata.obs['spatial_domain_18'] = adata.obs['spatial_domain_18'].astype('category')

We don't see a clear single peak, but we can see that the highest stability is for 18 clusters.

In [ ]:
adata.obs['spatial_domain_18'] = autok.predict(adata, use_rep='X_cellcharter', k=18)

In [ ]:
if 'spatial_domain_18_colors' in adata.uns:
    del adata.uns['spatial_domain_18_colors']

sq.pl.spatial_scatter(
    adata, 
    shape=None, 
    library_key='sample', 
    color=["spatial_domain_18", "cell_types"], 
    size=1,
    figsize=(10,10),
    title=np.repeat(adata.obs['sample'].cat.categories, 2),
    ncols=2
)

Now that we have the spatial domains, we can look at the cell type enrichment in each domain.

This function measures the likelihood of a cell type being found in a domain compared to random chance.

In [ ]:
cc.gr.enrichment(
    adata,
    group_key='spatial_domain_18',
    label_key='cell_types',
)
cc.pl.enrichment(
    adata,
    group_key='spatial_domain_18',
    label_key='cell_types',
    dot_scale=8
)

<span style="color: red;">**Task 4:**
The [Allen Brain Atlas](https://atlas.brain-map.org/atlas?atlas=1&plate=100960076#atlas=1&plate=100960520&resolution=6.98&x=5512.001546223959&y=3967.997233072917&zoom=-2) provides images and annotation of mouse brain samples at multiple depths.<br><br>
Choose one of the six samples, go to the atlas and try to find the image that better matches the regions shown by CellCharter.</span>

<span style="color: red;">1. Which of the 132 images is the closest one?</span><br>
<span style="color: red;">2. After identifying the closest image, try to match the domains obtained with the annotated regions in the atlas.</span><br>
<span style="color: red;">3. Do you find anatomical differences between the two conditions? are there differences in spatial domain compositions? Are there unique domains in one of the two conditions (Alzheimer vs wildtype)?</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- Focus, in particular on comparing samples from mice of the same age.</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- If you find difference between the two samples, what do you think is the cause?</span><br>


<span style="color: red;">**Task 5:** Now, save the spatial domains corresponding to 9 and 23 clusters into the `adata.obs` column `spatial_domain_9` and `spatial_domain_23`.</span>


In [ ]:
sq.pl.spatial_scatter(
    adata, 
    shape=None, 
    library_key='sample', 
    color=["spatial_domain_9", "spatial_domain_18", "spatial_domain_23"], 
    size=1,
    figsize=(10,10),
    title=np.repeat(adata.obs['sample'].cat.categories, 3),
    ncols=3
)

The three levels of clustering form a sort of hierarchical structure.<br>
9-cluster domains tend to separate into subdomains in the 18 clusters, and so on.

<span style="color: red;">**Task 6:** Go back to the Allen Brain Atlas and look if some of the hierarchical structure is reflected in the atlas.</span>


#### Shape characterization

Now, we are going to look at the shapes of the domains we obtained.

We first find the local components of the domains.
This is done by first finding the connected components of the spatial graph.

In [ ]:
cc.gr.connected_components(adata, cluster_key='spatial_domain_18', min_cells=100)

In [ ]:
# Hackfix: squidpy's spatial_scatter has some issues with categorical data with NaNs.
adata.obs['component_tmp'] = adata.obs['component'].astype('str')

In [ ]:
if 'component_tmp_colors' in adata.uns:
    del adata.uns['component_tmp_colors']

sq.pl.spatial_scatter(
    adata[(adata.obs['sample'].isin(['TgCRND8_17_9', 'TgCRND8_2_5']))], 
    shape=None, 
    library_key='sample', 
    color=["component_tmp", "spatial_domain_18"], 
    size=1,
    figsize=(10,10),
    title=np.repeat(['TgCRND8_17_9', 'TgCRND8_2_5'], 2),
    ncols=2
)

Then draw a boundary around every component

In [ ]:
cc.tl.boundaries(adata, alpha_start=10)

In [ ]:
cc.pl.boundaries(adata, sample='wildtype_5_7', show_cells=True)

And finally, by computing some shape metrics that we will use to compare the domains.

In [ ]:
cc.tl.curl(adata)
cc.tl.linearity(adata)

Let's look at the shape metrics of one of the cortex layers, domain 8.

In [ ]:
cc.pl.shape_metrics(adata, cluster_key='spatial_domain_18', figsize=(6,3), cluster_id=8, metrics=['curl', 'linearity'])

<span style="color: red;">**Task 7:** Look at the spatial domain 0.</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- Is it going to have lower, same or higher linearity than domain 8?</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- What about curl?</span>

<span style="color: red;">After you have answered, plot the shape metrics for domain 0 and check if it matches your expectations.</span>

Tip: you can pass multiple cluster ids to `plot_shape_metrics` in the form of a list to plot multiple domains at once.

<span style="color: red;">**Task 8:** Look at spatial domain 12.</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- What do you expect to see in terms of curl and linearity?</span><br>
<span style="color: red;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;- What about elongation?</span>

<span style="color: red;">After answering, compute the elongation and compare the shape metrics for domains 8 and 12.</span>

You can check [CellCharter's documentation](https://cellcharter.readthedocs.io/en/latest/tools.html) for the elongation metric.


## References

[1] Varrone, M., Tavernari, D., Santamaria-Martínez, A., Walsh, L. A. & Ciriello, G. CellCharter reveals spatial cell niches associated with tissue remodeling and cell plasticity. Nat Genet 56, 74–84 (2024).

[2] Singhal, V. et al. BANKSY unifies cell typing and tissue domain segmentation for scalable spatial omics data analysis. Nat Genet 56, 431–441 (2024).

[3] https://github.com/NBISweden/workshop-spatial/blob/main/labs/07b_spatial_domains.ipynb: Last access: 14.01.2025

[4] [ELIXIR 2025 Spatial transcriptomics course](https://github.com/elixir-europe-training/ELIXIR-SCO-spatial-omics/blob/main/day_3/practical_4/workdir/p4_niche_domain.ipynb). Last access: 9.10.2025